In [1]:


model_id = 'meta-llama/Meta-Llama-3-70B-Instruct'
model_name = 'Meta-Llama-3-70B-Instruct'

In [5]:
import huggingface_hub
from openai import OpenAI


def openai_chat(messages):
    client = OpenAI(
        base_url=f"https://api-inference.huggingface.co/models/{model_id}/v1/",
        #base_url='https://twkby3tkl3wdjrta.us-east-1.aws.endpoints.huggingface.cloud/v1/',
        api_key=huggingface_hub.get_token(),
    )

    return client.chat.completions.create(
        model=model_id,
        messages=messages,
        stream=False,
        max_tokens=1512
    )

In [6]:

import requests
import re


def api_key():
    with open(f"{os.environ['HOME']}/HuggingFace-API-DCU-AI.key", 'r') as file:
        return file.read().strip()


def api_url():
    #return f"https://api-inference.huggingface.co/models/{model_id}"
    return 'https://twkby3tkl3wdjrta.us-east-1.aws.endpoints.huggingface.cloud'


def api_headers():
    return {
        "Authorization": f"Bearer {api_key()}",
        "Content-Type": "application/json"
    }


def api_query(inputs):
    # https://huggingface.co/blog/inference-pro#controlling-text-generation
    # Looks like some params are for HF pro accounts
    payload = {
        "inputs": inputs,
        "parameters": {
            "max_new_tokens": 800,
            #"do_sample": True,
            "temperature": 0.7,
            #"top_p": 0.25,
            #"top_k": 40,
            # "repetition_penalty": 1.1,
            "return_full_text": False,
            "seed": 2024
        },
    }

    return requests.post(api_url(), headers=api_headers(), json=payload).json()


In [8]:
from huggingface_hub import InferenceClient
import os

client = InferenceClient(model=model_id, token=api_key())

In [9]:
def chat(messages):
    return client.chat_completion(
        model=model_id,
        messages=messages,
        temperature=0.8,
        stream=False,
        max_tokens=800
    ).choices[0].message.content

In [10]:
#import json
#import tiktoken

#encoding = tiktoken.encoding_for_model("gpt2")
#len(encoding.encode(json.dumps(c)))

In [60]:
import pandas as pd

path = 'augmented-taxonomies.parquet'

taxonomies = pd.read_parquet(path)
taxonomies = taxonomies[taxonomies['source'] != 'synthetic']
taxonomies['term'] = taxonomies['term'].str.lower()
# Remove duplicate rows in the 'term'  column
taxonomies = taxonomies.drop_duplicates(subset='term')
taxonomies['reason'] = taxonomies['reason'].str.replace('^synthetic:gpt-4o:', '', regex=True)
taxonomies = taxonomies[['source', 'category', 'term', 'reason']]
taxonomies

In [61]:
taxonomies['source'].value_counts()

In [62]:
taxonomies['category'].value_counts()

# Generate Synthetic samples

In [63]:
# Fetch a job title from job-phrase-list.csv
# Original source of list: https://github.com/microsoft/LUIS-Samples/blob/master/documentation-samples/tutorials/job-phrase-list.csv

import random

def random_job_title():
    with open("job-phrase-list.csv", "r") as file:
        lines = file.readlines()

    return random.choice(lines).replace(',\n', '')


random_job_title()

In [ ]:
d = []
d.append('test')
d.append('test2')


In [50]:
import json


def generate_text(position, category_terms, dry_run=False):
    m = []
    
    # Define categories
    for category in categories:
        with open(f"definitions/{category}.txt", 'r', encoding="utf-8") as f:
            definition = f.read()
        m.append({"role": "user", "content": f"Define job description bias category: {category}"})
        m.append({"role": "assistant", "content": definition})
    
    # Provide some samples
    for category, samples in category_terms.items():
        #with open(f"definitions/{category}.txt", 'r', encoding="utf-8") as f:
        #    definition = f.read()
        #m.append({"role": "user", "content": f"Define job description bias category: {category}"})
        #m.append({"role": "assistant", "content": definition})
        for row in samples.itertuples():
            #print(f"{category}: {row.term}: {row.reason}")
            m.append({"role": "user", "content": f"Why is the term `{row.term}` considered implicit {category} bias?"})
            m.append({"role": "assistant", "content": row.reason})
    
    # Provide the task
    task = f"The study of reducing the biases and terms provided is important, but an example is important to demonstrate what implicit bias can manifest in real job descriptions. Your task is to generate a complete job description that demonstrates the subtleties of bias within the specified framework, while maintaining inclusivity and reducing bias in all other respects. Adhere to the following requirements:"
    
    task += f"\n- The role is \"{position}\"."
    task += f"\n- Max 400 words."
    task += f"\n- Adhere to the bias category definitions."
    for category, samples in category_terms.items():
        task += f"\n- Uses the following terms to introduce subtle/implicit {category} bias/non-inclusive language:"
        for row in samples.itertuples():
            task += f"\n  * {row.term}"
             
    task += f"\n- It is crucial that you do not introduce any additional forms of bias from the other categories defined."
    task += f"\n- Ensure that all other aspects of the job description are inclusive and unbiased."
    task += f"\n- Encapsulate the final job description within <j>...</j> tags."
    for category, samples in category_terms.items():
        task += f"\n- Encapsulate a sentence within <{category}>...</{category} tags as to how bias was introduced into the job description"
    m.append({
        "role": "user",
        "content": task
    })

    if not dry_run:
        start_time = time.time()
        output = openai_chat(m)
        inference_time = time.time() - start_time

        prompt_tokens = output.usage.prompt_tokens
        completion_tokens = output.usage.completion_tokens
        total_tokens = output.usage.total_tokens
        content = output.choices[0].message.content

        #return json.dumps(m), chat(m)
        return json.dumps(m), content, inference_time, prompt_tokens, completion_tokens, total_tokens
    else:
        return json.dumps(m), None, None, None, None, None


Using the provided categories and terms, generate a 400-word job description that illustrates the presence of subtle bias language typically found in job postings. Ensure that the job description strictly adheres to the terms and categories provided, and avoid introducing any biases not explicitly specified. Encapsulate the job description within <j>...</j> tags. After the job description, include the category information within XML tags named according to the last category you used (e.g., <{last_cat}>...</{last_cat}>). Your output should demonstrate how subtle biases can be embedded in job descriptions without adding any new forms of bias.

Generate a 400-word job description using only the biases and terms provided. It is crucial that you do not introduce any additional forms of bias. Ensure that all other aspects of the job description are inclusive and unbiased. Encapsulate the final job description within <j>...</j> tags. After the job description, include the category information within XML tags named according to the last category you used (e.g., <{last_cat}>...</{last_cat}>). Your task is to demonstrate the subtleties of bias within the specified framework, while maintaining inclusivity and reducing bias in all other respects.

In [32]:
import math

test_data_size = 500
min_test_split_size = 0.05 

categories = ['age', 'disability', 'feminine', 'masculine', 'racial', 'sexuality', 'general']
splits = len(categories) + 1
size = 3000 #int(math.ceil((test_data_size / min_test_split_size) / splits)) 
max_additional_categories = 4  # Maximum number of additional categories per sample

label_categories = ['label_' + category for category in categories]
analysis_categories = ['analysis_' + category for category in categories]

In [44]:
import os
import pandas as pd

output_dir = '/home/teveritt/Datasets/2024-mcm-everitt-ryan/datasets/synthetic-job-postings/pass3'
output_file = f'{output_dir}/synthetic-biased-job-descriptions-sync.jsonl'

if os.path.exists(output_file):
    synthetic_df = pd.read_json(output_file, lines=True)
else:
    synthetic_df = pd.DataFrame(
        columns=["document_id", "position"] + label_categories + analysis_categories + ["inference_time", "prompt_tokens",
                                                                  "completion_tokens",
                                                                  "total_tokens", "text", "input", "output"])

synthetic_df

In [51]:
import time
import datetime
import pandas as pd
from itertools import cycle
import random
import math
import hashlib

categories_cycle = cycle(categories)

# Dictionary to keep track of the count of samples in each category
category_count = {category: 0 for category in categories}


def create_hash(input_string):
    return hashlib.sha256(input_string.encode()).hexdigest()[:10]


def extract_job_posting(text):
    content = re.findall(r'<j>(.*?)</j>', text, re.DOTALL)
    ret = [c.strip() for c in content]
    return ret[0] if len(ret) > 0 else text

def find_first_between_tags(file_content, tag):
    start_tag = f"<{tag}>"
    end_tag = f"</{tag}>"

    start_index = file_content.find(start_tag)
    end_index = file_content.find(end_tag)

    if start_index != -1 and end_index != -1:  # tags were found
        start_index += len(start_tag)  # adjust to index after the start tag
        result = file_content[start_index:end_index].strip()  # extract content between tags
        return result

    return None  # tags were not found or improperly formatted

dry_run = False

# Iterate through the categories in a round-robin fashion 
iter_size = size + len(categories)
for i in range(iter_size):
    category = next(categories_cycle)
    if not dry_run:
        print(f'{i}/{iter_size} Generating synthetic for category {category}')

    while category_count[category] >= size:
        # If the current category has already reached the size, 
        # we take the next category on the list
        category = next(categories_cycle)

    additional_categories = random.sample(categories, k=random.randint(0, max_additional_categories))
    additional_categories = set(additional_categories)
    additional_categories.add(category)

    # Don't include both to reduce confusing the model
    if 'masculine' in additional_categories and 'feminine' in additional_categories:
        additional_categories.remove('feminine')
        additional_categories.remove('masculine')
        additional_categories.add(random.choice(['masculine', 'feminine']))

    category_sample = int(math.ceil(max_additional_categories / len(additional_categories)))
    category_sample = random.randint(1, category_sample)
    category_terms = {}
    for cat in additional_categories:
        category_terms[cat] = taxonomies[taxonomies['category'] == cat].sample(category_sample)

    position = random_job_title()

    # Generate text sample with category information
    #prompt, output = generate_text(category_terms)
    prompt, output, inference_time, prompt_tokens, completion_tokens, total_tokens = generate_text(position,
                                                                                                   category_terms,
                                                                                                   dry_run)
    text = find_first_between_tags(output,'j') if not dry_run else None
    analysis_age = find_first_between_tags(output, 'age')
    analysis_disability = find_first_between_tags(output, 'disability')
    analysis_feminine = find_first_between_tags(output, 'feminine')
    analysis_masculine = find_first_between_tags(output, 'masculine')
    analysis_racial = find_first_between_tags(output, 'racial')
    analysis_sexuality = find_first_between_tags(output, 'sexuality')
    analysis_general = find_first_between_tags(output, 'general')
    
    
    category_count[category] += 1

    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    id = create_hash(output) if not dry_run else i
    id = f"{timestamp}:{id}"
    id_m = model_id.replace('/', ':')
    data = {
        "document_id": f'Synthetic:{id_m}:{id}',
        "position": position
    }
    for cat in categories:
        data[f'label_{cat}'] = False

    data['inference_time'] = inference_time
    data['prompt_tokens'] = prompt_tokens
    data['completion_tokens'] = completion_tokens
    data['total_tokens'] = total_tokens
    data['text'] = text
    data['analysis_age'] = analysis_age
    data['analysis_disability'] = analysis_disability
    data['analysis_feminine'] = analysis_feminine
    data['analysis_masculine'] = analysis_masculine
    data['analysis_racial'] = analysis_racial
    data['analysis_sexuality'] = analysis_sexuality
    data['analysis_general'] = analysis_general
    data['input'] = prompt
    data['output'] = output
    
    for cat in additional_categories:
        data[f'label_{cat}'] = True

    if not dry_run:
        with open(output_file, 'a') as file:
            if not os.stat(output_file).st_size == 0:
                file.write('\n')
            file.write(json.dumps(data))

    synthetic_df = pd.concat([synthetic_df, pd.DataFrame(data, index=[0])], ignore_index=True)
    break

synthetic_df

In [52]:
print(synthetic_df.tail(1)['text'].values[0])

In [58]:
i = """
This GPT is designed to detect implicit bias and non-inclusive language in job descriptions. It will analyse text for specific categories of bias and return two concise sentences (using British spelling and grammar) explaining the detected bias for each relevant category, without providing additional information. The possible categories of bias it can detect are: age, disability, feminine, general, masculine, racial, and sexuality.  Only list the ones detected, ignore the rest.  If none are detected, say "None detected".

Role and Goal: The GPT should identify implicit biases and non-inclusive language in job descriptions, providing a clear and succinct explanation of the bias detected for each relevant category. It should focus only on the labels detected and avoid extraneous information.

Constraints: The GPT should only output two sentences per detected bias and should not include additional commentary or information beyond the explanation of the bias. It should strictly only output the provided categories and adhere to the definitions provided for each category of bias.

Guidelines: The GPT should follow the specific examples and language cues given in the definitions for each category of bias. It should use precise and professional language in its explanations.

Clarification: The GPT should not ask for clarification but should make a best effort to analyze the text and provide accurate detections based on the provided definitions.  If no bias detected, say "None detected".

Personalization: The GPT should maintain a neutral, informative tone and focus on delivering clear, concise explanations of the detected biases.

Output format: For each category detected, wrap the sentences with the category tag.  For example, if racial, age, disability are detected then the output should be: <racial>The concise explanation here.</racial><age>The concise explanation here.</age><disability>The concise explanation here.</disability> 

Age bias definition: Occurs when language or requirements subtly favour certain age groups over others. Common categories include insensitive terms (e.g., "geezer"), language implying energy or modernity (e.g., "young and dynamic", "recent graduate") that favour younger candidates, as well as language implying experience and wisdom (e.g., "seasoned professional", "mature") that favour older candidates.

Disability bias definition: Involves the use of terms or requirements that inadvertently exclude or disadvantage individuals based on disabilities. This can include physical, mental, sensory, or cognitive impairments. Common categories include ableist terms that imply the requirement of a physical trait (e.g., "type 50 words per minute") instead of focusing on the job function (e.g., "enter data at 50 words per minute"), unnecessary physical requirements (e.g., "must be able to lift 50 pounds" for a desk job), and the absence of language regarding reasonable accommodations to ensure that candidates with disabilities are assessed based on their suitability for the role.

Feminine bias definition: Refers to language that subtly favours or resonates more with female candidates.  Common categories include gender-coded words (e.g., "nurturing," "supportive"), domestic or caregiving metaphors, an emphasis on collaborative over individualistic skills, and gendered job titles (e.g., "hostess") and pronouns (e.g., "she/her").

General bias definition: Occurs when language or requirements use derogatory (e.g. "feminazi", "retarded") or outdated terms (e.g. "the disabled"), or subtly favour or disadvantage candidates based on various characteristics. Common categories include socio-economic status (e.g., "blue-collar"), educational background (e.g., "Degree from a top school"), mental health (e.g., "OCD"), gender and family roles (e.g., "clean-shaven", "maternity leave"), veteran status, criminal history, and political or ideological beliefs.

Masculine bias definition: Refers to language that subtly favours or resonates more with male candidates. Common categories include gender-coded words (e.g., "dominant", "competitive"), sports or military metaphors, an emphasis on individualistic over collaborative skills, and gendered job titles (e.g., "salesman") and pronouns (e.g., "he/him").

Racial bias definition: Occurs when language or requirements subtly favour certain racial groups or exclude others. Common categories include racially insensitive terms (e.g., "master/slave", "redneck"), exclusionary phrases (e.g., "brown-bag session", "white/black list"), and assumptions about linguistic proficiency or background (e.g., "native English speaker").

Sexuality bias definition: Occurs when language or requirements subtly favour certain sexual orientations, gender identities, or expressions over others, creating non-inclusive language that can exclude LGBTQ+ individuals. Common categories include terms that enforce heteronormativity (e.g., "the men and women", "opposite sex"), outdated or offensive terminology (e.g., "homosexual", "tranny"), lack of recognition of diverse family structures (e.g., "wife and husband" instead of "partner" or "spouse"), assumptions about gender identity (e.g., "born a man", "sex change"), and non-inclusive pronouns (e.g., "he/she" instead of "they" or "you").
"""

m = [{
        "role": "system",
        "content": i
    },
    {
        "role": "user",
        "content": synthetic_df.head(1)['text'].values[0]
    }
]

openai_chat(m)

In [35]:
synthetic_df.to_parquet(f'{output_dir}/synthetic-biased-job-descriptions.parquet', compression='gzip')

In [36]:
for cat in categories:
    print(synthetic_df[f'label_{cat}'].value_counts())

In [43]:
synthetic_df[label_categories].tail(1)

In [38]:
print(synthetic_df['text'].iloc[-1])

In [207]:
# Longest phrase
longest_text = synthetic_df['text'].apply(lambda x: (len(x), x)).max()[1]
longest_text

In [208]:

from transformers import AutoTokenizer


def print_max_tokens(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)
    max_tokens = len(tokenizer.encode(longest_text))
    print(f"Max '{model_id}' tokens: {max_tokens}")

def print_encode_decoded(model_id, longest_text):
    tokenizer = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)
    encoded_tokens = tokenizer.encode(longest_text)
    print(f"Tokens: {encoded_tokens}")
    print(f"Decoded tokens: {tokenizer.decode(encoded_tokens)}")
    
def print_tokens(model_id, longest_text):
    tokenizer = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)
    tokens = tokenizer.tokenize(longest_text)
    print(f"Tokens: {tokens}")
    

In [209]:
max_char = len(longest_text)
max_words = len(longest_text.split())

print(f'Max characters: {max_char}')
print(f'Max words: {max_words}')
for model_id in ['roberta-base', 'bert-base-uncased', 'microsoft/deberta-v3-small']:
    print_max_tokens(model_id)


In [210]:
import numpy as np

# Source: https://colab.research.google.com/drive/1pddMaJJIHR0O8MND42hfzYRxOPMV82KA?usp=sharing#scrollTo=RkVuiK_loty4

def categorical_entropy(df, labels):
    # entropy for labels across the dataset
    # p(l) = count(l) / sum(count(l) for l in labels))
    # H = sum(p(l) * -log2 p(l) for l in labels)
    cat_sums = df[labels].sum()
    cat_probs = np.array([cs / cat_sums.sum() for cs in cat_sums])
    return np.sum(cat_probs * -np.log2(cat_probs))

In [211]:
label_categories

In [212]:
# entropy for original dataset
categorical_entropy(synthetic_df, label_categories)

In [214]:
dedup_df = pd.read_parquet('/home/teveritt/Datasets/2024-mcm-everitt-ryan/datasets/synthetic-jobs/synthetic-biased-job-descriptions-deduped.parquet')
dedup_df

In [215]:
categorical_entropy(dedup_df, label_categories)


In [216]:
for cat in categories:
    print(dedup_df[f'label_{cat}'].value_counts())

In [223]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

data = {}

for cat in categories:
    counts = dedup_df[f'label_{cat}'].value_counts()
    data[cat] = counts

df = pd.DataFrame(data)

plt.figure(figsize=(20, 16))  # set plot figure size here
df.plot(kind='bar', stacked=True)
plt.title('Distribution of the 7 Categories')
plt.xlabel('Categories')
plt.ylabel('Counts')
plt.show()

In [226]:
import matplotlib.pyplot as plt
import pandas as pd

data = {} 

for cat in categories:
    counts = dedup_df[f'label_{cat}'].value_counts()
    data[cat] = counts

df = pd.DataFrame(data)

plt.figure(figsize=(10, 8))

df.plot(kind='barh', stacked=True)

plt.title('Distribution of the 7 Categories')
plt.ylabel('Categories')
plt.xlabel('Counts')
plt.show()
